In [7]:
import h5py
import scanpy as sc
import anndata
import loompy as lp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
import glob

In [4]:
import sys
sys.path.insert(0, '/home/tchari/monod/src/')

In [5]:
import monod
#importlib.reload(monod)
from monod import preprocess, extract_data, cme_toolbox, analysis, mminference

In [71]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph

import igraph as ig
import leidenalg as la

In [44]:
# ********* Get meK_looms.tar.gz and meKMeans_data_samp_params.csv *********

samps = pd.read_csv('meKMeans_data_samp_params.csv')
samps.head()

,Dataset,Transcrip,Clusters,Cu,Lam
0,allen_b08,/home/tchari/perturbCME/notebooks/gg_200524_mo...,10,-6.777778,-1.25
1,allen_b02h01,/home/tchari/perturbCME/notebooks/gg_200524_mo...,11,-6.777778,-1.25
2,cl3,/home/tchari/perturbCME/notebooks/gg_200525_ge...,3,-6.777778,-1.25
3,cl5,/home/tchari/perturbCME/notebooks/gg_200525_ge...,5,-6.777778,-0.70
4,brca1,/home/tchari/perturbCME/notebooks/gg_200524_mo...,-1,-6.777778,-0.70


In [45]:
d_strings  = glob.glob('./hvg_objs/*.loom')
d_strings 

['./hvg_objs/allen_b08_300hvgs.loom',
 './hvg_objs/cl5_2000hvgs.loom',
 './hvg_objs/allen_b02h01_300hvgs.loom',
 './hvg_objs/cl5_300hvgs.loom',
 './hvg_objs/allen_b08_2000hvgs.loom',
 './hvg_objs/allen_b02h01_1000hvgs.loom',
 './hvg_objs/allen_b08_4000hvgs.loom',
 './hvg_objs/cl3_1000hvgs.loom',
 './hvg_objs/allen_b08_1000hvgs.loom',
 './hvg_objs/cl3_2000hvgs.loom',
 './hvg_objs/cl5_1000hvgs.loom',
 './hvg_objs/cl5_4000hvgs.loom',
 './hvg_objs/allen_b02h01_4000hvgs.loom',
 './hvg_objs/cl3_4000hvgs.loom',
 './hvg_objs/allen_b02h01_2000hvgs.loom',
 './hvg_objs/cl3_300hvgs.loom']

In [46]:
datas = pd.unique(samps.Dataset)
datas

array(['allen_b08', 'allen_b02h01', 'cl3', 'cl5', 'brca1', 'pbmc',
       'e11e13'], dtype=object)

In [68]:
attribute_names =[('unspliced','spliced'),'gene_name','barcode']

### **meK-Means**

Test with 2, correct K, K+5, K+10

In [ ]:
#For each loom in looms from ./hvg_objs

!mkdir /home/tchari/perturbCME/notebooks/fits/meKruns_0209

dir_strings = []
data_strings = []
result_strings = []

#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
meK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    #Inference params for sampling
    tech_tup = [list(samps.Cu[ind])[0],list(samps.Lam[ind])[0]]
    
    #Preprocessing params for filepaths
    trans = list(samps.Transcrip[ind])[0]
    k = list(samps.Clusters[ind])[0]
    data_name = list(samps.Dataset[ind])[0]
    
    
    ds = lp.connect(d)
    num_genes = len(ds.ra[attribute_names[1]])
    num_cells = len(ds.ca[attribute_names[2]])
    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' HVGs')
    
    Ks = [2,k,k+5,k+10]

    for clus in Ks:
        #Run Monod preprocessing to set up files for run, leave room for genes to be removed with low U/S
        dir_string,dataset_string = monod.preprocess.construct_batch([d], \
                                                     trans, \
                                                     [data_name], \
                                                     attribute_names=attribute_names,\
                                                     batch_location='./fits/meKruns_0209', \
                                                     meta=data_name+'_K'+str(clus), \
                                                     batch_id=1, \
                                                     n_genes=num_genes-60,exp_filter_threshold=None,viz=False)
        dir_strings += [dir_string]
        data_strings += [dataset_string]

        #Set model and run meK-Means
        
        #Define bounds for params
        phys_lb = [-2.0, -1.8, -1.8 ] 
        phys_ub = [4.2, 2.5, 2.5] 
        samp_lb = tech_tup 
        samp_ub = tech_tup  
        gridsize = [1,1] #Already have tech params
    

        epochs = 10

        # ---------------- meK-Means Inference ---------------- 
        #Define model with bursty transcription and Poisson molecule capture/sampling
        fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

        #Set up mminference parameters
        inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                    dataset_string[0],fitmodel,k=clus,epochs=epochs,use_lengths = True,
                    gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

        #Read in loom file with filtered barcodes
        search_data = monod.extract_data.extract_data(d, trans, data_name,
                    dataset_string[0], dir_string, viz=False, dataset_attr_names=attribute_names)

        #Run inference(fit_all_grid_points()) and Save result file strings
        full_result_string = inference_parameters.fit_all_grid_points(40,search_data) 

        result_strings.append(full_result_string)
        

        # ----------------  Save output ---------------- 
        #Read in results and get cluster assignments
        sr = [monod.analysis.load_search_results(i) for i in full_result_string]
        
        
        cat_assigns = np.array([None]*num_cells) #cat_assigns is empty list of length = no. cell barcodes
        for i in range(len(sr)):
            r = sr[i]
            cat_assigns[r.filt] = r.assigns #denotes which cluster (k), cells were assigned to

        print('No. clus: ',len(np.unique(cat_assigns)))
        

        meths += ['meK-Means']
        result += [cat_assigns]
        mats += ['U,S']
        hyper += [clus]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,cat_assigns)]
        ami += [adjusted_mutual_info_score(true_labs,cat_assigns)]


meK_res['Method'] = meths
meK_res['Clustering'] = result
meK_res['Matrices'] = mats
meK_res['Hyperparam'] = hyper
meK_res['HVGs'] = hvgs
meK_res['Data'] = data
meK_res['ARI'] = ari
meK_res['AMI'] = ami
    

allen_b08 with 299 HVGs


100%|████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.02s/it]


mstep self.weights:  [0.8630301 0.1369699]
Q Function:  -1169045.4176676944



100%|████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.29s/it]


mstep self.weights:  [0.87036381 0.12963619]
Q Function:  -1109316.3334968125



...

100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:47<00:00, 287.37s/it]


mstep self.weights:  [1.00000000e+00 2.02142713e-10]
Q Function:  -5103427.4975165585



100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:29<00:00, 269.10s/it]


mstep self.weights:  [1.00000000e+00 2.02142713e-10]
Q Function:  -5103427.4975165585



  0%|                                                                                                | 0/1 [00:00<?, ?it/s]

In [ ]:
#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, NMI
meK_res.to_csv('./fits/meKruns_0209/meK_bench_results.csv',index=None)
meK_res.head()


In [96]:
# dataset_string

['./fits/meKruns_0209/gg_240209_025_allen_b08_K2_1/allen_b08']

In [99]:
# !rm -r ./fits/meKruns_0209

### **Leiden & K-Means**


In [90]:
#Perform standard normalization (read-depth + log1p)
def norm_mat(mat,new_tot=1e4):
    '''
    mat: cell x gene matrix
    '''  
    tots = np.sum(mat,axis=1)
    divids = (new_tot/tots)[:,None]
    mat_norm = mat*divids
    mat_norm = np.log1p(mat_norm)
    
    mat_norm[np.isnan(mat_norm)] = 0
    
    return mat_norm

In [77]:
def get_graph(mat,neigh=30):
    A = kneighbors_graph(mat, neigh, mode='connectivity', include_self=True)
    sources, targets = A.nonzero()
    weights = A[sources, targets]
    if isinstance(weights, np.matrix):
        weights = weights.A1
    g = ig.Graph(directed=False) #True in scanpy, which is odd
    g.add_vertices(A.shape[0])  # this adds adjacency.shape[0] vertices
    g.add_edges(list(zip(sources, targets)))

    g.es['weight'] = weights
    
    return g

In [91]:
# ! rm -r /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0209

In [ ]:
#Make the U,S, U+S, Concat matrices, res 0.75,1,1.5,2

!mkdir /home/tchari/perturbCME/notebooks/fits/LeidKMeansruns_0209


#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, AMI
LK_res = pd.DataFrame()
meths = []
result = []
mats = []
hyper = []
hvgs = []
data = []
ari = []
ami = []

for d in d_strings:
    
    #Get tech param by keyword (datas) in loom paths
    ind = [i in d for i in datas]
    
    k = list(samps.Clusters[ind])[0]
    data_name = list(samps.Dataset[ind])[0]
    
    
    ds = lp.connect(d)
    num_genes = len(ds.ra[attribute_names[1]])
    num_cells = len(ds.ca[attribute_names[2]])
    U = ds.layers[attribute_names[0][0]][:,:].T
    S = ds.layers[attribute_names[0][1]][:,:].T
    true_labs = list(ds.ca['subclass_label'])
    ds.close()
    print(data_name+' with '+str(num_genes)+' HVGs')
    
    res = [0.75,1,1.5,2]
    
    #Get U,S,U_p_S,U_c_S
        
    #Norm mats
    U_norm = norm_mat(U)
    S_norm = norm_mat(S)
    U_p_S_norm = norm_mat(U+S)
    U_c_S_norm = norm_mat(np.concatenate([U,S],axis=1))
    
    g_U = get_graph(U_norm)
    g_S = get_graph(S_norm)
    g_U_p_S = get_graph(U_p_S_norm)
    g_U_c_S = get_graph(U_c_S_norm)
    
    # -------- Fit Leiden -------
    for r in res:
        
        partition_type = la.RBConfigurationVertexPartition
        #Make Knn graphs
        for g in [g_U,g_S,g_U_p_S,g_U_c_S]:
            part = la.find_partition(g, partition_type, 
                                        weights=np.array(g.es['weight']).astype(np.float64), 
                                        n_iterations=-1, resolution_parameter=r)

            labels = np.array(part.membership)
            
            # ----- Save Results ---- ARI,AMI etc
            meths += ['Leiden']
            result += [labels]
            hyper += [r]
            hvgs += [num_genes]
            data += [data_name]
            ari += [adjusted_rand_score(true_labs,labels)]
            ami += [adjusted_mutual_info_score(true_labs,labels)]
            
#             print('No. clus Leiden: ',len(np.unique(labels)))
            
        mats += ['U','S','U+S','UcatS']
        
        
        
    # -------- Fit K-Means with k -------
    
    for x in [U_norm,S_norm,U_p_S_norm,U_c_S_norm]:
        kmeans = KMeans(n_clusters=k, random_state=0).fit(x)
        labs = kmeans.labels_
        
        meths += ['K-Means']
        result += [labs]
        hyper += [k]
        hvgs += [num_genes]
        data += [data_name]
        ari += [adjusted_rand_score(true_labs,labs)]
        ami += [adjusted_mutual_info_score(true_labs,labs)]
        
#         print('No. clus K-Means: ',len(np.unique(labs)))
        
    mats += ['U','S','U+S','UcatS']

    
LK_res['Method'] = meths
LK_res['Clustering'] = result
LK_res['Matrices'] = mats
LK_res['Hyperparam'] = hyper
LK_res['HVGs'] = hvgs
LK_res['Data'] = data
LK_res['ARI'] = ari
LK_res['AMI'] = ami

In [93]:
#Save method, clustering, mat_in (U,S, etc), hyper (K or res), hvgs, data, ARI, NMI
LK_res.to_csv('./fits/LeidKMeansruns_0209/LeidKMeans_bench_results.csv',index=None)
LK_res.head()



,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
0,Leiden,"[1, 3, 5, 1, 0, 7, 2, 8, 3, 1, 0, 2, 1, 3, 3, ...",U,0.75,299,allen_b08,0.701710,0.832537
1,Leiden,"[1, 2, 6, 1, 0, 8, 3, 9, 2, 10, 0, 3, 1, 2, 2,...",S,0.75,299,allen_b08,0.675147,0.849593
2,Leiden,"[1, 2, 6, 1, 0, 8, 3, 9, 2, 10, 0, 3, 1, 2, 2,...",U+S,0.75,299,allen_b08,0.662079,0.836243
3,Leiden,"[4, 1, 7, 4, 0, 9, 3, 10, 1, 2, 0, 3, 2, 1, 1,...",UcatS,0.75,299,allen_b08,0.570947,0.802221
4,Leiden,"[4, 2, 5, 4, 0, 8, 1, 9, 2, 3, 0, 1, 3, 2, 2, ...",U,1.00,299,allen_b08,0.617054,0.799360
